In [1]:
import timeit
import numpy as np
import pandas as pd
from scipy import stats
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
sns.set_style('whitegrid')
plt.style.use('seaborn-whitegrid')

__author__ = 'HK Dambanemuya'
__version__ = 'Python 3'


Bad key "text.kerning_factor" on line 4 in
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# K-Fold cross validation
def run_cv(X,y,clf_class):
    # Construct a kfolds
    kf = KFold(n_splits=5,shuffle=True)
    kf.get_n_splits(X)
    y_pred = y.copy()

    # Iterate through folds
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

# Evaluation Summary
def evaluation_summary(df_map):
    eval_df = pd.DataFrame()
    t_, mean_, std_ = [], [], []
    for k, v in tqdm_notebook(df_map.items()):
        if k[-2:] == '00':
            threshold = f"1{k[-2:]}%"
        else:
            threshold = f"{k[-2:]}%"
        
        t_.append(threshold[:-1])
        
        X = v[v.columns[1:-1]].values
        y = v[v.columns[-1]].values
        
        auc_ = [] 
        
        start = timeit.default_timer()
        auc_ = [auc(y, run_cv(X,y,clf)) for i in tqdm_notebook(range(2)) ]    
        stop = timeit.default_timer()
        
        mean_.append(np.mean(auc_))
        std_.append(np.std(auc_))
        
        print (f"Completed 2 iterations of {threshold} dataset in {stop-start} seconds. AUC (mean, std) = {np.mean(auc_)}, {np.std(auc_)}.")
    
    eval_df['Threshold'] = t_
    eval_df['Mean'] = mean_
    eval_df['Std'] = std_
    
    return eval_df

# AUC Metric
def auc(y_true, y_pred): return metrics.roc_auc_score(y_true, y_pred)

# Initialize MinMax Scaler
scaler = MinMaxScaler() 

# Initialize Random Forest Classifier
clf = RandomForestClassifier(random_state=42)

In [3]:
# Import Bid Data
df_map = {'bids_1': pd.read_csv("../../Data/lending/aggregated/bids_1.csv").fillna(-1),
        'bids_10': pd.read_csv("../../Data/lending/aggregated/bids_10.csv").fillna(-1)}

In [4]:
eval_df = evaluation_summary(df_map)
eval_df.to_csv('../../Data/evaluation/lending.csv', index=False)
eval_df


Completed 2 iterations of _1% dataset in 148.167524664 seconds. AUC (mean, std) = 0.637069853795979, 6.891805502429182e-05.



Completed 2 iterations of 10% dataset in 161.62265068699998 seconds. AUC (mean, std) = 0.7560089015734792, 0.0004955686123500613.



,Threshold,Mean,Std
0,_1,0.637,0.000
1,10,0.756,0.000
